In [1]:
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv('lemm123gramsHealthWellness.csv', encoding='unicode_escape')


In [3]:
data.shape


(614, 52614)

In [4]:
data.head()


,Rating,X058,X058.078,X058.078.098,X078,X078.098,X078.098.shaker,X098,X098.shaker,X098.shaker.depending,...,zero.star.option,zero.star.somehow,zipped,zipped.chair,zipped.chair.stripped,zipped.didnt,zipped.didnt.see,zumba,zumba.qi,zumba.qi.gong
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
np.random.seed(123)
data0 = data.reindex(np.random.permutation(data.index))


In [6]:
data1 = data0.iloc[:,3:]


In [7]:
data1.shape


(614, 52611)

In [8]:
target=data0.iloc[:,0:1]


In [9]:
target.shape


(614, 1)

In [10]:
print(target['Rating'].unique())
print(len(target['Rating'].unique()))


[5 4 3 2 1]
5


In [11]:
mean_vals = np.mean(data1, axis=0)
std_val = np.std(data1)

data1_centered = (data1 - mean_vals)/std_val

print(data1_centered.shape, target.shape)



(614, 52611) (614, 1)


In [12]:
print(data1.head())


     X058.078.098  X078  X078.098  X078.098.shaker  X098  X098.shaker  \
583             0     0         0                0     0            0   
11              0     0         0                0     0            0   
443             0     0         0                0     0            0   
442             0     0         0                0     0            0   
267             0     0         0                0     0            0   

     X098.shaker.depending  X10  X10.copay  X10.copay.kaiser  ...  \
583                      0    0          0                 0  ...   
11                       0    0          0                 0  ...   
443                      0    0          0                 0  ...   
442                      0    0          0                 0  ...   
267                      0    0          0                 0  ...   

     zero.star.option  zero.star.somehow  zipped  zipped.chair  \
583                 0                  0       0             0   
11            

In [13]:
print(target.head())


     Rating
583       5
11        4
443       5
442       3
267       4


In [14]:
class_mapping = {label: idx for idx, label in enumerate(np.unique(target['Rating']))}
class_mapping


{1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

In [15]:
target['OH_rating']=target['Rating']
target['OH_rating'] = target['Rating'].map(class_mapping)
target.head()


C:\Users\m\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\m\Anaconda2\envs\python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Rating,OH_rating
583,5,4
11,4,3
443,5,4
442,3,2
267,4,3


In [16]:
target1 = target['OH_rating']
target1.head()


583    4
11     3
443    4
442    2
267    3
Name: OH_rating, dtype: int64

In [17]:
X_train = data1[:491]
X_test = data1[491:]
y_train = target1[:491]
y_test = target1[491:]

################################
# for adding the names of the classes after prediction from earlier in script
y_trainNames = target['Rating']
y_trainNames = y_trainNames[:491]
y_trainNames.columns=['Rating']
y_trainNames1=pd.DataFrame(y_trainNames)

y_testNames = target['Rating']
y_testNames = y_testNames[491:]
y_testNames.columns=['Rating']
y_testNames1=pd.DataFrame(y_testNames)
################################

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(491, 52611)
(491,)
(123, 52611)
(123,)


In [18]:
y_train


583    4
11     3
443    4
442    2
267    3
      ..
319    4
364    4
412    4
141    4
454    3
Name: OH_rating, Length: 491, dtype: int64

In [19]:
import tensorflow as tf
import tensorflow.contrib.keras as keras
#optionally use import tensorflow.keras as keras when no longer experimental contributor package development

np.random.seed(123)
tf.set_random_seed(123)


In [20]:
model6 = keras.models.Sequential()

model6.add(
    keras.layers.Dense(
        units=20000,   #output units need to match next layer inputs 
        input_dim=52611, #number of features for input above says 52611
        kernel_initializer='glorot_uniform',# name of the guy behind Xavier Initialization; the biases to zero
        bias_initializer='zeros',
        activation='tanh'))

model6.add(
    keras.layers.Dense(
        units=10000,   #output matches next layer input 
        input_dim=20000, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))

model6.add(
    keras.layers.Dense(
        units=500,   #output matches next layer input 
        input_dim=10000, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='tanh'))


model6.add(
    keras.layers.Dense(
        units=400,   #output matches next layer input 
        input_dim=500, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'))


model6.add(
    keras.layers.Dense(
        units=200,   #output matches next layer input 
        input_dim=400, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'))


model6.add(
    keras.layers.Dense(
        units=100,   #output matches next layer input 
        input_dim=200, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'))



model6.add(
    keras.layers.Dense(
        units=100,   #output matches next layer input 
        input_dim=100, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'))

model6.add(
    keras.layers.Dense(
        units=100,   #output matches next layer input 
        input_dim=100, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'))


model6.add(
    keras.layers.Dense(
        units=100,   #output matches next layer input 
        input_dim=100, #input matches last layer's output
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='relu'))


model6.add(
    keras.layers.Dense(
        units=19,  #these are the number of class categories in our target  
        input_dim=100,
        kernel_initializer='glorot_uniform',
        bias_initializer='zeros',
        activation='softmax'))#will return the class membership probs summing to 1 of all class probs

# these are hyperparameters that can be tuned if overfitting during training, or to get better accuracy
sgd_optimizer = keras.optimizers.SGD( 
        lr=0.01, decay=1e-5, momentum=1.5)

# categorical_crossentropy is used in multiclass classification instead of binary_crossentropy
# to match the softmax function
model6.compile(optimizer=sgd_optimizer,
              loss='sparse_categorical_crossentropy')
# it was 'categorical_crossentropy', but that expects binary matrices of 1s and 0s
# it said to use sparse_categorical_crossentropy


W0506 17:13:54.184705 10796 deprecation.py:506] From C:\Users\m\Anaconda2\envs\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
import time
start=time.time()

history6 = model6.fit(X_train, y_train,
                    batch_size=50, epochs=5,
                    verbose=1, 
                    validation_split=0.1) 
end=time.time()
fit_time=(end-start)
print(start,end,fit_time)


Train on 441 samples, validate on 50 samples
Epoch 1/5
441/441 [==============================] - 17526s 40s/sample - loss: 2.7434 - val_loss: 1.5892
Epoch 2/5
250/441 [================>.............] - ETA: 10:13:12 - loss: 3.7715

In [ ]:
y_train_pred6 = model6.predict_classes(X_train, verbose=0)
print('First 3 predictions: ', y_train_pred6[:3])


In [ ]:
y_train_pred6 = model6.predict_classes(X_train, 
                                     verbose=0)


In [ ]:
y_train_pred6 = pd.DataFrame(y_train_pred6)
y_train_pred6.columns=['predicted']

y_train6 = y_train
y_train6 = pd.DataFrame(y_train6)
y_train6.columns=['OH_Rating']

y_train_pred6.index=y_train6.index

Train6=pd.concat([y_train6['OH_Rating'],y_trainNames1['Rating'],y_train_pred6['predicted']],axis=1)

print(Train6)


In [ ]:
y_test_pred6 = model6.predict_classes(X_test, 
                                    verbose=0)



In [ ]:
y_test_pred6 = pd.DataFrame(y_test_pred6)
y_test_pred6.columns=['predicted']

y_test6 = y_test
y_test6 = pd.DataFrame(y_test6)
y_test6.columns=['OH_Rating']

y_test_pred6.index=y_test6.index

Test6=pd.concat([y_test6['OH_Rating'],y_testNames1['Rating'],y_test_pred6['predicted']],axis=1)

print(Test6)


In [ ]:
s = sum(Train6['OH_Rating']==Train6['predicted'])
l = len(Train6['Rating'])
accTrain6 = s/l
print('Training Correctly Predicted:',s,'Training Accuracy:',accTrain6,'\n')


In [ ]:
s = sum(Test6['OH_Rating']==Test6['predicted'])
l = len(Test6['OH_Rating'])
accTest6 = s/l
print('Testing Correctly Predicted:',s,'Testing Accuracy:',accTest6)
